In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

In [2]:
df = pd.read_csv("poetry.csv")

In [3]:
pd.set_option('display.max_colwidth', None)
df.head()

,Unnamed: 0,Title,Poem,Poet,Tags
0,0,\r\r\n Objects Used to Prop Open a Window\r\r\n,"\r\r\nDog bone, stapler,\r\r\ncribbage board, garlic press\r\r\n because this window is loose—lacks\r\r\nsuction, lacks grip.\r\r\nBungee cord, bootstrap,\r\r\ndog leash, leather belt\r\r\n because this window had sash cords.\r\r\nThey frayed. They broke.\r\r\nFeather duster, thatch of straw, empty\r\r\nbottle of Elmer's glue\r\r\n because this window is loud—its hinges clack\r\r\nopen, clack shut.\r\r\nStuffed bear, baby blanket,\r\r\nsingle crib newel\r\r\n because this window is split. It's dividing\r\r\nin two.\r\r\nVelvet moss, sagebrush,\r\r\nwillow branch, robin's wing\r\r\n because this window, it's pane-less. It's only\r\r\na frame of air.\r\r\n",Michelle Menting,NaN
1,1,\r\r\n The New Church\r\r\n,"\r\r\nThe old cupola glinted above the clouds, shone\r\r\namong fir trees, but it took him an hour\r\r\nfor the half mile all the way up the hill. As he trailed,\r\r\nthe village passed him by, greeted him,\r\r\nasked about his health, but everybody hurried\r\r\nto catch the mass, left him leaning against fences,\r\r\nmeasuring the road with the walking stick he sculpted.\r\r\nHe yearned for the day when the new church\r\r\nwould be built—right across the road. Now\r\r\nit rises above the moon: saints in frescoes\r\r\nmeet the eye, and only the rain has started to cut\r\r\nthrough the shingles on the roof of his empty\r\r\nhouse. The apple trees have taken over the sky,\r\r\nsequestered the gate, sidled over the porch.\r\r\n",Lucia Cherciu,NaN
2,2,\r\r\n Look for Me\r\r\n,"\r\r\nLook for me under the hood\r\r\nof that old Chevrolet settled in weeds\r\r\nat the end of the pasture.\r\r\nI'm the radiator that spent its years\r\r\nbolted in front of an engine\r\r\nshoving me forward into the wind.\r\r\nWhatever was in me in those days\r\r\nhas mostly leaked away,\r\r\nbut my cap's still screwed on tight\r\r\nand I know the names of all these\r\r\ntattered moths and broken grasshoppers\r\r\nthe rest of you've forgotten.\r\r\n",Ted Kooser,NaN
3,3,\r\r\n Wild Life\r\r\n,"\r\r\nBehind the silo, the Mother Rabbit\r\r\nhunches like a giant spider with strange calm:\r\r\nsix tiny babies beneath, each\r\r\nclamoring for a sweet syringe of milk.\r\r\nThis may sound cute to you, reading\r\r\nfrom your pulpit of plenty,\r\r\nbut one small one was left out of reach,\r\r\na knife of fur\r\r\nbarging between the others.\r\r\nI watched behind a turret of sand. If\r\r\nI could have cautioned the mother rabbit\r\r\nI would. If I could summon the\r\r\nBunnies to fit him in beneath\r\r\nthe belly's swell\r\r\nI would. But instead, I stood frozen, wishing\r\r\nfor some equity. This must be\r\r\nwhy it's called Wild Life because of all the\r\r\ncrazed emotions tangled up in\r\r\nthe underbrush within us.\r\r\nDid I tell you how\r\r\nthe smallest one, black and trembling,\r\r\nhopped behind the kudzu\r\r\nstill filigreed with wanting?\r\r\nShould we talk now of animal heritage, their species,\r\r\ncreature development? And what do we say\r\r\nabout form and focus—\r\r\nwriting this when a stray goes hungry, and away.\r\r\n",Grace Cavalieri,NaN
4,4,\r\r\n Umbrella\r\r\n,"\r\r\nWhen I push your button\r\r\nyou fly off the handle,\r\r\nold skin and bones,\r\r\nblack bat wing.\r\r\nWe're alike, you and I.\r\r\nBoth of us\r\r\nresemble my mother,\r\r\nso fierce in her advocacy\r\r\non behalf of\r\r\nthe most vulnerable child\r\r\nwho'll catch his death\r\r\nin this tempest.\r\r\nSuch a headwind!\r\r\nSometimes it requires\r\r\nall my strength\r\r\njust to end a line.\r\r\nBut when the wind is at\r\r\nmy back, we're likely\r\r\nto get carried away, and say\r\r\nsomething we can never retract,\r\r\nsomething saturated from the ribs\r\r\ndown, an old stony\r\r\nword like ruin. You're what roof\r\r\nI have, frail thing,\r\r\nyou're my argument\r\r\nagainst the whole sky.\r\r\nYou're the fundamental difference\r\r\nbetween wet and dry.\r\r\n",Connie Wanek,NaN


In [4]:
df.isna().sum()

Unnamed: 0      0
Title           0
Poem            0
Poet            0
Tags          955
dtype: int64

In [5]:
df = df.drop(['Unnamed: 0', 'Tags', 'Poet'], axis=1)

In [6]:
df['Title'] = df['Title'].str.replace('\r', '')
df['Poem'] = df['Poem'].str.replace('\r', '')
df['Title'] = df['Title'].str.replace('\n', '')
df['Poem'] = df['Poem'].str.strip('\n')

In [7]:
# Lower case everything
df['Title'] = df['Title'].str.lower()
df['Poem'] = df['Poem'].str.lower()

# Remove apostrophes and join the parts
df['Title'] = df['Title'].str.replace("'", "", regex=False)
df['Poem'] = df['Poem'].str.replace("'", "", regex=False)

In [8]:
# Remove tokens containing numbers from the Titles
def remove_numeric_tokens(text):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.isalpha()]
    processed_text = ' '.join(filtered_tokens)
    return processed_text

df['Title'] = df['Title'].apply(remove_numeric_tokens)

In [9]:
# Filter out rows where both 'Title' and 'Poem' have no alphabetic characters
df = df[df.apply(lambda x: any(c.isalpha() for c in x['Title']) and 
                            any(c.isalpha() for c in x['Poem']), axis=1)]

In [10]:
df.tail()

,Title,Poem
13829,january,"the wise men will unlearn your name.\nabove your head no star will flame.\none weary sound will be the same—\nthe hoarse roar of the gale.\nthe shadows fall from your tired eyes\nas your lone bedside candle dies,\nfor here the calendar breeds nights\ntill stores of candles fail.\nwhat prompts this melancholy key?\na long familiar melody.\nit sounds again. so let it be.\nlet it sound from this night.\nlet it sound in my hour of death—\nas gratefulness of eyes and lips\nfor that which sometimes makes us lift\nour gaze to the far sky.\nyou glare in silence at the wall.\nyour stocking gapes: no gifts at all.\nits clear that you are now too old\nto trust in good saint nick;\nthat its too late for miracles.\n—but suddenly, lifting your eyes\nto heavens light, you realize:\nyour life is a sheer gift."
13831,the bean eaters,"they eat beans mostly, this old yellow pair. \ndinner is a casual affair.\nplain chipware on a plain and creaking wood, \ntin flatware.\ntwo who are mostly good.\ntwo who have lived their day,\nbut keep on putting on their clothes \nand putting things away.\nand remembering ...\nremembering, with twinklings and twinges,\nas they lean over the beans in their rented back room that is full of beads and receipts and dolls and cloths, tobacco crumbs, vases and fringes."
13832,the spider,"i \nthe spider expects the cold of winter.\nwhen the shadows fall in long autumn\nhe congeals in a nest of paper, prepares\nthe least and minimal existence,\nobedient to nature. no other course\nis his; no other availed him when\nin high summer he spun and furled\nthe gaudy catches. i am that spider,\ncaught in nature, summer and winter.\nyou are the symbol of the seasons too.\n \nii \nnow to expatiate and temporize\nthis artful brag. i never saw so quieting\na sight as the dawn, dew-clenched foot-\nwide web hung on summer barn-eaves, spangled.\nit moves to zephyrs that is tough as steel.\ni never saw so finely-legged a creature\nwalk so accurate a stretch as he,\nproud, capable, patient, confident.\nto the eye he gave close penetration\ninto real myth, the myth of you, of me.\n \niii \nyet, by moving eyesight off from this\nthere is another dimension. near the barn,\ndown meadow to shingle, no place for spiders,\nthe sea in large blue breathes in brainstorm tides,\npirates itself away to ancient spain,\npirouettes past purgatory to paradise.\ndo i feed deeper on a spider,\na close-hauled view upon windless meaning,\nor deeper a day or dance or doom bestride\non ocean’s long reach, on parables of god?"
13833,the distances,"the accumulation of reefs\npiling up one over the others\nlike thoughts of the sky increasing as the head rises\nunto horizons of wet december days perforated\nwith idle motions of gulls . . . and our feelings.\n \ni’ve been wondering about what you mean,\nstanding in the spray of shadows before an ocean\nabandoned for winter, silent as a barque of blond hair.\n \nand the way the clouds are bending, the way they “react”\nto your position, where your hands close over your breasts\nlike an eyelid approving the opening of “an evening’s light.”\n \nparasites attach themselves to the moss covering\nyour feet, blind cubans tossing pearls across the jetty,\nand the sound of blood fixes our eyes on the red waves.\n \n it is a shark!\n \nand our love is that rusted bottle . . . pointing north,\nthe direction which we turn, conjuring up our silver knives\nand spoons and erasing messages in the sand, where you wrote\n \n“freezing in the arctic of our dreams,” and i said\n“yes” delaying the cold medium for a time\nwhile you continued to “cultivate our possessions”\n \nas the moon probably “continued” to cradle.\ntan below the slant of all those wasted trees\nwhile the scent carried us back to where we were:\n \ndancing like the children of great diplomats\nwith our lean bodies draped in bedsheets and\nleather flags while the orchestra made sounds\n \nwhich we thought was the sky, but was only a series\

In [11]:
# Make newline a special token
df['Poem'] = df['Poem'].str.replace('\n', ' <N> ')

In [12]:
df.isna().sum()

Title    0
Poem     0
dtype: int64

In [13]:
tokenized_titles = df['Title'].apply(word_tokenize)
# Only consider tokens without numbers in them as  I observed many tokens had numbers in them which to me will just serve as noise
all_words = [word for title_words in tokenized_titles for word in title_words if word.isalpha()]
unique_words = list(set(all_words))
# For unknown tokens(after deployment) and padding
special_tokens = ["<UNK>", "<PAD>"]
unique_words = special_tokens + unique_words
print(len(unique_words))

10911


In [14]:
tokenized_titles

0        [objects, used, to, prop, open, a, window]
1                                [the, new, church]
2                                   [look, for, me]
3                                      [wild, life]
4                                        [umbrella]
                            ...                    
13829                                     [january]
13831                           [the, bean, eaters]
13832                                 [the, spider]
13833                              [the, distances]
13848                                     [january]
Name: Title, Length: 13619, dtype: object

In [15]:
df['Title'].size

13619

In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Define hyperparameters (adjust as needed)
vocab_size = len(unique_words)  # Number of unique words in the vocabulary
embedding_dim = 105  # Number of dimensions for word embeddings
context_window_size = 3  # Number of surrounding words considered for context (adjust)
batch_size = 64
learning_rate = 0.01
device = "cuda"
# Class for the CBOW-based word embedding model
class WordEmbedder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_window_size):
        super().__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.context_window_size = context_window_size
        
    def forward(self, titles):
        batch_size, total_length = titles.shape
        pad_index = 1  # Assuming padding index is 1
        
        # Initialize an empty list to hold the context embeddings
        all_context_embeddings = []
    
        for batch in range(batch_size):
            title = titles[batch]
    
            # Find indices that are not padding
            non_pad_indices = (title != pad_index).nonzero().squeeze()
            
            # Ensure non_pad_indices is a 1D tensor for indexing
            if non_pad_indices.dim() == 0:
                non_pad_indices = non_pad_indices.unsqueeze(0)  # Make it 1D
    
            # Calculate start and end indices for slicing the title
            start = non_pad_indices[0] - self.context_window_size if non_pad_indices[0] >= self.context_window_size else 0
            end = non_pad_indices[-1] + self.context_window_size + 1 if non_pad_indices[-1] + self.context_window_size + 1 <= total_length else total_length
            
            # Adjusted title with minimal necessary padding
            adjusted_title = title[start:end]
    
            # Now, for each word in the adjusted title, calculate the context embedding
            for i in range(self.context_window_size, len(adjusted_title) - self.context_window_size):
                context_indices_left = adjusted_title[i-self.context_window_size:i]
                context_indices_right = adjusted_title[i+1:i+1+self.context_window_size]
                context_indices = torch.cat((context_indices_left, context_indices_right), dim=0).unsqueeze(0)
    
                # Compute and append the average context embedding
                context_embedding = self.embeddings(context_indices).mean(dim=1)
                all_context_embeddings.append(context_embedding)
    
        # Concatenate all context embeddings into a single tensor for processing
        all_context_embeddings = torch.cat(all_context_embeddings, dim=0)
    
        return all_context_embeddings

In [17]:
embedding_model = WordEmbedder(vocab_size, embedding_dim, context_window_size).to(device)
print(embedding_model)

WordEmbedder(
  (embeddings): Embedding(10911, 100)
)


In [18]:
longest_length = 0
longest_title = None

for title in tokenized_titles:
    if len(title) > longest_length:
        longest_length = len(title)
        longest_title = title

print("Length of the longest title:", longest_length)
print("The longest title:", ' '.join(longest_title))

Length of the longest title: 35
The longest title: within two weeks the african american poet ross gay is mistaken for both the african american poet terrance hayes and the african american poet kyle dargan not one of whom looks anything like the others


In [19]:
# We map word to indexes instead of ohe as it takes up lesser space and is faster
word_to_index = {word: idx for idx, word in enumerate(unique_words)}
# Replace all words in the titles with its respective tokens indexes, isb its not in the word_to_index dictionary, replace with 0(index of <UNK>) 
indexed_titles = [[word_to_index.get(word, word_to_index["<UNK>"]) for word in title] for title in tokenized_titles]

# Define the padding value
pad_token = word_to_index["<PAD>"]  
# Pad the shorter lists with pad_token to match the length of the longest list
padded_titles = []
for title in indexed_titles:
    # We define padding
    padding = [pad_token] * context_window_size  
    # Padding on both sides
    padded_title = padding + title + padding
    # Pad until all lists are of equal length
    while len(padded_title) < longest_length + (2 * context_window_size): 
        padded_title.append(pad_token)
    padded_titles.append(padded_title)

# Convert padded_titles to a PyTorch tensor
indexed_titles_tensor = torch.tensor(padded_titles)

# Convert input/padded_titles into proper format
title_dataset = TensorDataset(indexed_titles_tensor)
data_loader = DataLoader(title_dataset, batch_size=64, shuffle=True)

In [20]:
# Initialize our optimizer and loss function
optimizer = torch.optim.Adam(embedding_model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [21]:
for i in data_loader:
    print(i[0].shape)
    break

torch.Size([64, 41])


In [22]:
word_to_index["<UNK>"]

0

In [23]:
titles_with_unk = [title for title in indexed_titles if 0 in title]
titles_with_unk

[]

In [24]:
num_epochs = 15  # Set the number of epochs

for epoch in range(num_epochs):
    total_loss = 0.0
    for batch in data_loader:  # Assuming batch is a tensor of titles
        titles = batch[0].to(device)  # Extract titles if DataLoader returns a tuple

        optimizer.zero_grad() 
        # Forward pass: compute predicted context embeddings for all titles in the batch
        predicted_context_embeddings = embedding_model(titles)
        
        # A simplified assumption: every non-pad token in `titles` is a target for which
        # the model has produced a context embedding. This may need adjusting based on the model.
        non_pad_mask = titles != pad_token
        target_embeddings = embedding_model.embeddings(titles[non_pad_mask]).to(device)
        loss = loss_fn(predicted_context_embeddings, target_embeddings)

        # Backward pass: compute gradient and update weights
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Print average loss for the epoch
    average_loss = total_loss / len(data_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}')

Epoch [1/15], Average Loss: 0.7817
Epoch [2/15], Average Loss: 0.3672
Epoch [3/15], Average Loss: 0.2386
Epoch [4/15], Average Loss: 0.1677
Epoch [5/15], Average Loss: 0.1222
Epoch [6/15], Average Loss: 0.0908
Epoch [7/15], Average Loss: 0.0685
Epoch [8/15], Average Loss: 0.0522
Epoch [9/15], Average Loss: 0.0399
Epoch [10/15], Average Loss: 0.0307
Epoch [11/15], Average Loss: 0.0237
Epoch [12/15], Average Loss: 0.0182
Epoch [13/15], Average Loss: 0.0141
Epoch [14/15], Average Loss: 0.0109
Epoch [15/15], Average Loss: 0.0084


In [42]:
# Now that we have successfully trained our word embedding layer, time to make our positional matrix, which we will make using the formula mentioned in the paper
def positional_encoding(batch_size, max_seq_length, d_model):
    # Initialize the positional encoding matrix
    pe = torch.zeros(batch_size, max_seq_length, d_model)
    
    # Compute the positional encoding values
    position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(0).unsqueeze(-1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
    
    # Compute sine and cosine components
    pe[:, :, 0::2] = torch.sin(position * div_term)
    pe[:, :, 1::2] = torch.cos(position * div_term)
    
    return pe

In [43]:
pos_encoding = positional_encoding(batch_size, longest_length + (2 * context_window_size), embedding_dim)
pos_encoding.shape

torch.Size([64, 41, 100])

In [45]:
# Time to make the Multi-head Self Attention block!
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super().__init__()
        self.heads = heads

        # 3 Linear Layers for Q, K and V
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)
        
        # Since the feature or embedding dimension is typically the last dimension
        self.softmax = nn.Softmax(dim=-1)

        # Last Linear layer for the attention
        self.w_a = nn.Linear(embedding_dim, embedding_dim)
        
    def forward(self, embedding_vector):
        batch_size, seq_len, embedding_dim = embedding_vector.size()
        # Compute Q, K, and V
        Q = self.w_q(embedding_vector)
        K = self.w_k(embedding_vector)
        V = self.w_v(embedding_vector)

        # Seperate into heads
        head_dim = embedding_dim // self.heads
        Q = Q.view(batch_size, self.heads, seq_len, head_dim)
        K = K.view(batch_size, self.heads, seq_len, head_dim)
        V = V.view(batch_size, self.heads, seq_len, head_dim)

        # Calculate attention
        attention = torch.matmul(self.softmax(torch.matmul(Q, K.transpose(-1, -2)) / torch.sqrt(embedding_dim)), V)

        # Concatenating the attention heads (Transposing for correct concatenation)
        attention = attention.transpose(1, 2).view(batch_size, seq_len, embedding_dim)
        output = self.w_a(attention)
        return output

In [52]:
Q = torch.rand(2, 3, 2, 4)
Q

tensor([[[[0.5948, 0.4629, 0.9841, 0.9810],
          [0.7569, 0.4444, 0.8599, 0.1573]],

         [[0.0410, 0.2651, 0.7164, 0.1736],
          [0.5461, 0.4083, 0.2160, 0.2952]],

         [[0.2096, 0.1985, 0.7276, 0.8832],
          [0.0461, 0.4797, 0.6174, 0.1708]]],


        [[[0.3996, 0.4926, 0.4556, 0.4357],
          [0.0740, 0.8970, 0.2265, 0.8611]],

         [[0.1757, 0.5160, 0.1248, 0.7196],
          [0.0583, 0.8802, 0.3027, 0.4638]],

         [[0.1885, 0.5968, 0.4710, 0.7351],
          [0.0434, 0.9290, 0.6689, 0.9437]]]])

In [59]:
Q.transpose(1, 2).reshape(2, 2, -1)

tensor([[[0.5948, 0.4629, 0.9841, 0.9810, 0.0410, 0.2651, 0.7164, 0.1736,
          0.2096, 0.1985, 0.7276, 0.8832],
         [0.7569, 0.4444, 0.8599, 0.1573, 0.5461, 0.4083, 0.2160, 0.2952,
          0.0461, 0.4797, 0.6174, 0.1708]],

        [[0.3996, 0.4926, 0.4556, 0.4357, 0.1757, 0.5160, 0.1248, 0.7196,
          0.1885, 0.5968, 0.4710, 0.7351],
         [0.0740, 0.8970, 0.2265, 0.8611, 0.0583, 0.8802, 0.3027, 0.4638,
          0.0434, 0.9290, 0.6689, 0.9437]]])

In [53]:
Q.view(2, 2, 12)

tensor([[[0.5948, 0.4629, 0.9841, 0.9810, 0.7569, 0.4444, 0.8599, 0.1573,
          0.0410, 0.2651, 0.7164, 0.1736],
         [0.5461, 0.4083, 0.2160, 0.2952, 0.2096, 0.1985, 0.7276, 0.8832,
          0.0461, 0.4797, 0.6174, 0.1708]],

        [[0.3996, 0.4926, 0.4556, 0.4357, 0.0740, 0.8970, 0.2265, 0.8611,
          0.1757, 0.5160, 0.1248, 0.7196],
         [0.0583, 0.8802, 0.3027, 0.4638, 0.1885, 0.5968, 0.4710, 0.7351,
          0.0434, 0.9290, 0.6689, 0.9437]]])